In [15]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM, Input, Attention, Flatten, Concatenate
from keras.optimizers import SGD
import matplotlib.pyplot as plt

In [16]:
# Load data
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('../data/input/intents.json').read())

In [26]:
# Preprocess data
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('../data/output/Attention/words.pkl', 'wb'))
pickle.dump(classes, open('../data/output/Attention/classes.pkl', 'wb'))


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/codespace/nltk_data'
    - '/home/codespace/.python/current/nltk_data'
    - '/home/codespace/.python/current/share/nltk_data'
    - '/home/codespace/.python/current/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [23]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)

In [24]:
# Separate train_x and train_y
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

In [25]:
# Define model architecture
input_layer = Input(shape=(len(train_x[0]),))
embedding_layer = Embedding(input_dim=len(words), output_dim=100)(input_layer)
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(embedding_layer)
lstm_layer = Bidirectional(LSTM(256, return_sequences=True))(lstm_layer)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Apply Dropout for regularization
dropout_layer = Dropout(0.5)(lstm_layer)

In [ ]:
# Flatten the output
flatten_layer = Flatten()(dropout_layer)


In [ ]:
# Add Dense layers
dense_layer = Dense(128, activation='relu')(flatten_layer)
dense_layer = Dense(64, activation='relu')(dense_layer)

In [ ]:
# Output layer
output_layer = Dense(len(train_y[0]), activation='softmax')(dense_layer)


In [ ]:
# Define model
model = Model(inputs=[input_layer], outputs=[output_layer])

In [ ]:
# Compile model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(train_x, train_y, epochs=7700, batch_size=100, validation_split=0.1)


KeyboardInterrupt: 

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Save the model
model.save('../data/output/Attention/Rasika.model')

# Print "Done" after training
print("Done")